In [46]:
import logging
import pathlib

import numpy as np
import pandas as pd
from textblob import TextBlob

In [68]:
data = pathlib.Path.cwd() / 'data'
pkls = data / 'pkls'
npys = data / 'npys'
log_file = data / 'logs' / 'corrections.log'
log_file.touch()

fmt = '{name} - {asctime} - {levelname} - Message: {message}'

logging.basicConfig(filename=log_file, level=logging.INFO, style='{', format=fmt)
logger = logging.getLogger(__name__)

In [81]:
def correct_range(name, start=0, stop=None):
    pkl = pkls / f'{name}.pkl'
    npy = npys / f'{name}_corrections.npy'

    df = pd.read_pickle(pkl)
    essays = df['essay'][start:stop]
    
    if npy.exists():
        arr = np.load(npy)
    else:
        arr = np.empty(len(essays), dtype=object)

    logger.info(f'Start Index: {start}')
    
    corrs = (str(TextBlob(essay).correct()) for essay in essays)
    try:
        for i, corr in enumerate(corrs, start=start):
            arr[i] = corr
            logger.info(f'Corrected Index: {i}')
            
    except BaseException as exc:
        logger.exception(exc)
        raise
    finally:
        np.save(npy, arr)
        logger.info(f'Saved Corrections')
    
    logger.info(f'Stop Index: {stop}')

In [88]:
correct_range('valid', 7)

KeyboardInterrupt: 